# Analyzing Borrower Default Risk in Credit Scoring

## Introduction

In the competitive landscape of banking and finance, accurate credit assessment is crucial for mitigating risks associated with borrower defaults. This project aims to investigate the relationship between borrowers' marital status, the number of children they have, and the probability of defaulting on loans. By analyzing existing credit eligibility data, we seek to provide insights that can inform credit evaluations and improve decision-making processes within the bank's credit division.

### Goals

To achieve this, we have established the following research objectives:

1. **Hypothesis 1:** Investigate whether there is a correlation between having children and the probability of defaulting on a loan.
2. **Hypothesis 2:** Explore the correlation between family status and the probability of defaulting on a loan.
3. **Hypothesis 3:** Analyze the relationship between income level and the probability of defaulting on a loan.
4. **Hypothesis 4:** Examine how different credit goals influence the default percentage.

### Data Description

The analysis will be conducted using a dataset that includes the following attributes:

- **`children`:** Number of children in the family.
- **`days_employed`:** Customer work experience in days.
- **`dob_years`:** Customer age in years.
- **`education`:** Customer's education level.
- **`education_id`:** Identifier for the customer's education level.
- **`family_status`:** Marital status of the customer.
- **`family_status_id`:** Identifier for the customer's marital status.
- **`gender`:** Customer gender.
- **`income_type`:** Job type of the customer.
- **`debt`:** Whether the customer has ever defaulted on a loan.
- **`total_income`:** Monthly income of the customer.
- **`purpose`:** The purpose of obtaining a loan.

The analysis will involve several key steps, beginning with data preprocessing to ensure the dataset's integrity. This includes identifying and addressing missing values, correcting data types, and eliminating duplicates. Following this, we will explore the relationships between borrower characteristics—such as family status, income levels, and loan purposes—and their impact on default probabilities. The findings will be synthesized into a comprehensive report, serving as a valuable resource for assessing potential borrowers and refining credit policies.

Through this project, we aim to contribute to the development of more effective credit scoring methodologies that can enhance financial stability for both the bank and its clients.

## Import Data and Understand the information

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
try:
    # Try loading the file from your laptop path
    data = pd.read_csv('C:/Users/Eugene/Documents/GitHub/TripleTen-Projects/2. Bank Credit Scoring Analysis/credit_scoring_eng.csv')
except FileNotFoundError:
    # If the file is not found, try loading from the PC path
    data = pd.read_csv('C:/Users/user/OneDrive/Documents/GitHub/TripleTen-Projects/2. Bank Credit Scoring Analysis/credit_scoring_eng.csv')

In [3]:
data.shape

(21525, 12)

In [4]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
print(len(data))

21525


Check the missing value in the current data

In [7]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

From the current data, there are some missing value and it have the symmetric values. So verbally it can be said that the amount of missing data is correlated with each other. But it is necessary to check further whether the missing data is the same. Hence we check the relationship for the missing values from both columns

In [8]:
data[(data['days_employed'].isnull()) & (data['total_income'].isnull())].shape

(2174, 12)

In [9]:
total = data.shape[0]

missing = data.isnull().sum().sum()

percentage_missing = (missing / total) * 100

print(f'Missing value percentage: {percentage_missing:.2f}%')

Missing value percentage: 20.20%


**Temporary conclusion** 

The number of rows in the table that has been filtered has the same value as the number of missing values, namely 2174. The percentage of missing data is 20.20%. If the missing data has a weak correlation with the analysis results, the missing data can be *dropped* or deleted from the data used. However, if the data has a correlation that can affect the analysis, then the data will be cleaned by means of data manipulation and transformation.

Check for customers who have no data on the identified characteristics and columns with missing values

In [10]:
missing_characteristic_columns = ['days_employed', 'total_income']

In [11]:
for column in missing_characteristic_columns:
    missing_data = data[data[column].isnull()]
    distribution = missing_data['income_type'].value_counts()
    print(f"Distribution of customers who do not have {column} data based on job type:")    
    print(distribution, "\n")


Distribution of customers who do not have days_employed data based on job type:
income_type
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: count, dtype: int64 

Distribution of customers who do not have total_income data based on job type:
income_type
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: count, dtype: int64 



In [12]:
characteristic_columns = ['children','days_employed','dob_years','education_id','family_status_id','debt','total_income']

for column in characteristic_columns:
    missing_data = data[data[column].isnull()]
    distribution = missing_data['income_type'].value_counts()
    print(f"Distribution of customers who do not have {column} data based on job type:")    
    print(distribution, '\n')

Distribution of customers who do not have children data based on job type:
Series([], Name: count, dtype: int64) 

Distribution of customers who do not have days_employed data based on job type:
income_type
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: count, dtype: int64 

Distribution of customers who do not have dob_years data based on job type:
Series([], Name: count, dtype: int64) 

Distribution of customers who do not have education_id data based on job type:
Series([], Name: count, dtype: int64) 

Distribution of customers who do not have family_status_id data based on job type:
Series([], Name: count, dtype: int64) 

Distribution of customers who do not have debt data based on job type:
Series([], Name: count, dtype: int64) 

Distribution of customers who do not have total_income data based on job type:
income_type
employee         1105
business          508
retiree           413
civil servant     147
entrepr

**Temporary conclusion** 

From the distribution of the dataset above, the results are similar to the distribution that has been filtered. This is evidence for missing data patterns in the data. To deal with missing values ​​in the data, data of the numeric type can use median or average values ​​to fill in the blank data, and for categorical data types, the mode of the data column can be used.


## Data transformation

### Column Transformation

#### `education` Column

Let's look at all the values ​​in the education column to check what spelling needs to be corrected

In [13]:
unique_education_values = data['education'].unique()
print(unique_education_values)

["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


In [14]:
data['education'] = data['education'].str.lower()

Recheck all the values ​​in the column to ensure that we have corrected them appropriately

In [15]:
updated_education_value = data['education'].unique()
print(updated_education_value)

["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


#### `children` Column

In [16]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [17]:
data['children'].value_counts()

children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: count, dtype: int64

In [20]:
total_children = len(data['children'])

#-1
a = len(data[data['children']== -1])

#20
b = len(data[data['children']== 20])

persentase_a = (a/total_children) * 100
persentase_b = (b/total_children) * 100

print(f'Percentage value -1: {persentase_a:.2f}%')

print(f'Percentage value 20: {persentase_b:.2f}%')

Percentage value -1: 0.22%
Percentage value 20: 0.35%


From the distribution of the number of children in the data, there are irregularities in the number of children. such as '-1' in the number of children. The value 20 also includes incorrect data and can be caused by errors during data entry.

In [21]:
data.loc[data['children'] == -1, 'children'] = None
data.loc[data['children'] == 20, 'children'] = None

In [22]:
data['children'].value_counts()

children
0.0    14149
1.0     4818
2.0     2055
3.0      330
4.0       41
5.0        9
Name: count, dtype: int64

#### `days_employed` Column

Find problematic data in the `days_employed` column if there is a problem and calculate the percentage

In [23]:
data['days_employed'].head(15)

0      -8437.673028
1      -4024.803754
2      -5623.422610
3      -4124.747207
4     340266.072047
5       -926.185831
6      -2879.202052
7       -152.779569
8      -6929.865299
9      -2188.756445
10     -4171.483647
11      -792.701887
12              NaN
13     -1846.641941
14     -1844.956182
Name: days_employed, dtype: float64

Look for problematic data in the 'days_employed' column (large negative or positive values) then calculate the percentage of problematic data

In [24]:
problematic_data = data[(data['days_employed'] < 0) | (data['days_employed'] > 20000)]

percentage_problematic = (len(problematic_data) / len(data)) * 100

print("The amount of problematic data:", len(problematic_data))
print("Percentage of problematic data: {:.2f}%".format(percentage_problematic))

The amount of problematic data: 19351
Percentage of problematic data: 89.90%


The percentage of problematic data can be said to be high, with a value of 89.9%. One way to overcome this, is to change large negative values ​​to positive and ignore large positive values, as this can allow errors to occur when the input and data become inconsistent.

Resolve problematic values, if it is exist and check redistribution after changes

In [25]:
data['days_employed'] = data['days_employed'].apply(lambda x: -x if x < 0 else x)

data['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

The use of `apply` above is used to change negative values ​​to positive, after that I print the statistical distribution with `.describe`.

In [26]:
data['days_employed'].head(15)

0       8437.673028
1       4024.803754
2       5623.422610
3       4124.747207
4     340266.072047
5        926.185831
6       2879.202052
7        152.779569
8       6929.865299
9       2188.756445
10      4171.483647
11       792.701887
12              NaN
13      1846.641941
14      1844.956182
Name: days_employed, dtype: float64

Check the results - make sure that the problem is fixed

In [29]:
data['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

In [30]:
problematic_data = data[(data['days_employed'] < 0) | (data['days_employed'] > 20000)]

percentage_problematic = (len(problematic_data) / len(data)) * 100

print("The amount of problematic data:", len(problematic_data), "\n")
print("Percentage of problematic data: {:.2f}%".format(percentage_problematic), "\n")


The amount of problematic data: 3445 

Percentage of problematic data: 16.00% 



#### `dob_years` Column

Check `dob_years` for suspicious values ​​and calculate the percentage

In [31]:
suspicious_dob = data[(data['dob_years'] == 0) | (data['dob_years'] > 100)]

suspicious_percentage = len(suspicious_dob) / len(data) * 100

print("Amount of suspicious data:", len(suspicious_dob))
print("Percentage of suspicious data:", suspicious_percentage, "%")

Amount of suspicious data: 101
Percentage of suspicious data: 0.4692218350754936 %


In [32]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

The value that is doubtful is the value 0. Because the data could be invalid or wrong. Hence troubleshoot the `dob_years` column, if there is a problem by calculate the median age value from valid data and replace the value 0 with the median age.

In [33]:
median_age = data[data['dob_years'] > 0]['dob_years'].median()

data['dob_years'] = data['dob_years'].replace(0, median_age)

Replace the value 0 with the median value and recheck the issue.

In [34]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int64)

#### `family_status` Column

In [35]:
data['family_status'].unique()


array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [36]:
data['family_status'].head(15)

0               married
1               married
2               married
3               married
4     civil partnership
5     civil partnership
6               married
7               married
8     civil partnership
9               married
10              married
11              married
12    civil partnership
13              married
14    civil partnership
Name: family_status, dtype: object

#### `gender` Column

In [37]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [38]:
data['gender'].head(15)

0     F
1     F
2     M
3     M
4     F
5     M
6     F
7     M
8     F
9     M
10    M
11    F
12    M
13    F
14    F
Name: gender, dtype: object

Resolve problematic values, if it is exist, which value 'XNA' and replace it with value 'Unknown'

In [39]:
data.loc[data['gender'] == 'XNA','gender'] = 'Unknown'

In [40]:
data['gender'].unique()

array(['F', 'M', 'Unknown'], dtype=object)

#### `income_type` Column

In [41]:
data['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

In [42]:
data['income_type'].head(15)

0     employee
1     employee
2     employee
3     employee
4      retiree
5     business
6     business
7     employee
8     employee
9     employee
10    business
11    employee
12     retiree
13    employee
14    business
Name: income_type, dtype: object

### Handle Duplicate Data

Check for duplicates

In [43]:
duplicate_rows = data[data.duplicated()]
print("Number of duplicate data: ", duplicate_rows.shape[0])

Number of duplicate data:  71


In [44]:
duplicate_rows.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0.0,NaN,41,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family
3290,0.0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
4182,1.0,NaN,34,bachelor's degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony
4851,0.0,NaN,60,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony
5557,0.0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding


Since there are some duplicated data, it must be resolved by getting rid of it

In [45]:
data = data.drop_duplicates()

In [48]:
data[data.duplicated()].shape[0]

0

Do a final check to check if we have any duplicates

In [49]:
duplicate_rows = data[data.duplicated()]
print("Number of duplicate data: ", duplicate_rows.shape[0])


Number of duplicate data:  0


In [50]:
duplicate_rows.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Check the size of the dataset you now have after the first manipulation you performed

In [51]:
data.shape

(21454, 12)

There is a lot of inconsistent data in the dataset, whether numerical or categorical. This inconsistent data has been resolved in various ways, such as replacing it with median numbers and for categorical data, the sentences have been changed to one sentence that is used again (no repetition, such as BACHELOR, bachelor, etc.). In this data, there is also some duplicate data, and this data has been resolved by 'dropping'. Even so, there is still data that is consistent and does not need to be transformed.

### Handle Missing Value

In [52]:
data.isnull().sum()

children             123
days_employed       2103
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2103
purpose                0
dtype: int64

#### Handle `total_income` missing value

Handle the missing value in `total_income` by age category

In [53]:
def categorize_age(age):
    if age < 30:
        return "young"
    elif 30 <= age <= 45:
        return "middle-aged"
    else:
        return "old"
    
data.loc[:,'age_category'] = data['dob_years'].apply(categorize_age)

Perform testing to see whether your function works or not

In [54]:
data['age_category'].value_counts()

age_category
old            9150
middle-aged    9124
young          3180
Name: count, dtype: int64

Create a table with no missing values ​​and display some of its rows to make sure everything works fine

In [55]:
data_clean = data.dropna(subset=['total_income'])

Note the average value for income based on the factors you have identified

In [56]:
income_mean_by_education = data_clean.groupby('age_category')['total_income'].mean()
income_mean_by_education

age_category
middle-aged    28408.242913
old            25605.151757
young          25533.960641
Name: total_income, dtype: float64

Pay attention to the median value for income based on the factors you have identified

In [57]:
income_median_by_income_type = data_clean.groupby('age_category')['total_income'].median()
income_median_by_income_type

age_category
middle-aged    24687.5035
old            22112.4450
young          22742.6535
Name: total_income, dtype: float64

After the median and average values ​​have been obtained, we can look at the distribution and see whether there are extreme values ​​that affect the average value. If the data distribution contains several outliers and is not evenly distributed, using the median value may be more suitable than the average value.

For this case, the value will use the average value because there is no significant difference between the median and the average.

Create the function we will use to fill in the missing values

In [58]:
def fill_missing_income(row):
    age_category = row['age_category']
    if age_category == 'young':
        return 25533.96
    elif age_category == 'middle-aged':
        return 28408.24
    else:
        return 25605.15

Apply the function to each row

In [59]:
data['total_income'].fillna(data.apply(fill_missing_income, axis=1), inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_25496\924135103.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['total_income'].fillna(data.apply(fill_missing_income, axis=1), inplace=True)


Check if we get any errors

In [60]:
print("Number of lost values after filling:", data['total_income'].isnull().sum())

Number of lost values after filling: 0


In [61]:
data.isnull().sum()

children             123
days_employed       2103
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_category           0
dtype: int64

#### Handle `days_employed` missing value

Median distribution of `days_employed` based on the parameters you identified

In [62]:
median_days_employed_by_income_type = data.groupby('income_type')['days_employed'].median()
median_days_employed_by_income_type

income_type
business                         1547.382223
civil servant                    2689.368353
employee                         1574.202821
entrepreneur                      520.848083
paternity / maternity leave      3296.759962
retiree                        365213.306266
student                           578.751554
unemployed                     366413.652744
Name: days_employed, dtype: float64

Average distribution of `days_employed` based on the parameters you identified

In [63]:
mean_days_employed_by_income_type = data.groupby('income_type')['days_employed'].mean()
mean_days_employed_by_income_type

income_type
business                         2111.524398
civil servant                    3399.896902
employee                         2326.499216
entrepreneur                      520.848083
paternity / maternity leave      3296.759962
retiree                        365003.491245
student                           578.751554
unemployed                     366413.652744
Name: days_employed, dtype: float64

For this case, missing data will use the median value. This is because there are extreme values ​​in the retiree category and these values ​​will significantly influence the average value. Hence we write a function that calculates the median based on the identified parameters.

In [64]:
def fill_missing_days_employed(df):
    median_days_employed = df.groupby('income_type')['days_employed'].median()

    def fill_median_by_income_type(row):
            if pd.isnull(row['days_employed']):
                return median_days_employed[row['income_type']]
            return row['days_employed']
    
    df['days_employed'] = df.apply(fill_median_by_income_type, axis=1)

    return df

In [65]:
data = fill_missing_days_employed(data)

In [66]:
data.isnull().sum()

children            123
days_employed         0
dob_years             0
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt                  0
total_income          0
purpose               0
age_category          0
dtype: int64

#### Handle `children` missing value

In [67]:
data['children'].unique()

array([ 1.,  0.,  3.,  2., nan,  4.,  5.])

In [68]:
data['children'] = data['children'].fillna(0)

In [69]:
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_category        0
dtype: int64

## Data categorization

In the data categorization section, it will be categorized data based on `purpose`, `dob_years`, `income_type`, `debt`, `total_income`.

`purpose`: to find out what the customer's purpose is in applying for a loan

`dob_years`: current customer age

`income_type`: income type from work

`debt`: whether the customer has any ongoing debt

`total_income`: the amount of income received by the customer.

In [70]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int64)

In [71]:
data['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

In [72]:
data['debt'].unique()

array([0, 1], dtype=int64)

In [73]:
data['total_income'].unique()

array([40620.102, 17932.802, 23341.752, ..., 14347.61 , 39054.888,
       13127.587])

In [74]:
data['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Grouping process by creating a function for the categorization based on general topics

In [75]:
def categorize_data(row):
    age = row['dob_years']
    purpose = row['purpose']
    income_type = row['income_type']
    debt = row['debt']
    
    # Age Category
    if age < 35:
        return 'Young'
    elif age >= 35 and age <= 55:
        return 'Middle-aged'
    else:
        return 'Old'
    
    # Loan Purpose Category
    if 'properti' in purpose or 'real estate' in purpose:
        return 'Property'
    elif 'education' in purpose:
        return 'Education'
    elif 'car' in purpose:
        return 'Car'
    elif 'wedding' in purpose:
        return 'Wedding'
    else:
        return 'Others'
    
    # Income Type Category
    if income_type == 'employee':
        return 'Employee'
    elif income_type == 'retiree':
        return 'Retiree'
    elif income_type == 'business':
        return 'Business'
    elif income_type == 'civil servant':
        return 'Civil servant'
    elif income_type == 'unemployed':
        return 'Unemployed'
    elif income_type == 'entrepreneur':
        return 'Entrepreneur'
    elif income_type == 'student':
        return 'Student'
    elif income_type == 'paternity / maternity leave':
        return 'Maternity Leave/Paternity Leave'
    
    # Risk Category

    if income_type == 'employee':
        if debt == 0:
            return 'Low'
        else:
            return 'Middle'
    else:
        return 'High'

Create a column containing categories and calculate the values

In [76]:
data['category'] = data.apply(categorize_data, axis = 1)

View all numeric data in the selected column for categorization

In [78]:
numeric_data_dob_years = data['dob_years']

statistics_dob_years = numeric_data_dob_years.describe()

numeric_data_debt = data['debt']

statistics_debt = numeric_data_debt.describe()

numeric_data_total_income = data['total_income']

statistics_total_income = numeric_data_total_income.describe()

Get a statistical conclusion for the column

In [79]:
print("Statistical Conclusion for Column 'dob_years':")
statistics_dob_years

Statistical Conclusion for Column 'dob_years':


count    21454.000000
mean        43.473665
std         12.213507
min         19.000000
25%         33.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [80]:
print("Statistical Conclusion for Column 'debt':")
statistics_debt

Statistical Conclusion for Column 'debt':


count    21454.000000
mean         0.081150
std          0.273072
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: debt, dtype: float64

In [82]:
print("Statistical Conclusion for Column 'total_income':")
statistics_total_income

Statistical Conclusion for Column 'total_income':


count     21454.000000
mean      26786.703522
std       15653.176537
min        3306.762000
25%       17219.817250
50%       24967.101500
75%       31330.237250
max      362496.645000
Name: total_income, dtype: float64

## Hypothesis Testing

### Hypothesis 1: Is there a correlation between having children and the probability of defaulting on a loan?

In [83]:
total_customers_by_children = data.groupby('children')['children'].count()

customers_with_debt_by_children = data.groupby('children')['debt'].sum()

children_debt_data = pd.concat([total_customers_by_children, customers_with_debt_by_children], axis=1)
children_debt_data.columns = ['Total Customers', 'Customers with Debt']

In [84]:
children_debt_data['Debt Percentage'] = (children_debt_data['Customers with Debt'] / children_debt_data['Total Customers']) * 100

children_debt_data

,Total Customers,Customers with Debt,Debt Percentage
children,,,
0.0,14214,1072,7.541860
1.0,4808,444,9.234609
2.0,2052,194,9.454191
3.0,330,27,8.181818
4.0,41,4,9.756098
5.0,9,0,0.000000


From the analysis above, the smallest percentage is held by people who do not have children with a value of 7.54%, followed by people who have 3 children with a percentage of 8.18%. The analysis above can also be seen that the trend is that people who have a large number of children tend to default on loans at a higher rate than people who have no children or with fewer children. Although it must be underlined that the difference in percentage is not very significant.

### Hypothesis 2: Is there a correlation between family status and the probability of defaulting on a loan

In [85]:
family_debt_percentage = data.groupby('family_status')['debt'].mean().reset_index()

In [86]:
family_debt_percentage['debt'] = family_debt_percentage['debt'] * 100

family_debt_percentage

,family_status,debt
0,civil partnership,9.347145
1,divorced,7.112971
2,married,7.545182
3,unmarried,9.750890
4,widow / widower,6.569343


From the analysis above, people who hold the status of "unmarried" and "civil partnership" have a higher failure percentage with figures of 9.75% and 9.35% respectively. For people who hold the "widow/widower" status, the default percentage is lower at 6.57%. People who hold the status "divorced" and "married" have a relatively low failure percentage with figures of 7.11% and 7.545%. Family status in this analysis may influence the probability of loan default, but other factors also need to be considered to get more accurate results.

### Hypothesis 3: Is there a correlation between income level and the probability of defaulting on a loan?

In [87]:
unique_income = data['total_income'].unique().tolist()

unique_income.sort()

unique_income

[3306.762,
 3392.845,
 3418.824,
 3471.216,
 3503.298,
 3595.641,
 3815.153,
 3913.227,
 4036.463,
 4049.374,
 4212.77,
 4245.348,
 4386.4,
 4444.179,
 4465.254,
 4494.861,
 4592.45,
 4650.812,
 4664.644,
 4672.012,
 4708.271,
 4759.97,
 4812.103,
 4818.545999999999,
 4860.001,
 4919.749,
 5002.295,
 5028.623,
 5029.439,
 5037.321,
 5045.56,
 5053.838,
 5090.55,
 5112.186,
 5137.573,
 5148.514,
 5167.9940000000015,
 5168.082,
 5172.669,
 5195.285,
 5208.353,
 5217.0340000000015,
 5220.544,
 5259.254,
 5274.611,
 5288.165,
 5290.465,
 5330.769,
 5331.621,
 5335.014,
 5402.85,
 5409.738,
 5430.683000000001,
 5443.908,
 5452.4940000000015,
 5461.996,
 5464.092,
 5478.583000000001,
 5490.018,
 5496.834,
 5514.581,
 5515.539000000002,
 5529.334,
 5531.2040000000015,
 5562.874,
 5577.521,
 5579.965,
 5591.44,
 5595.911999999998,
 5604.991999999998,
 5622.0790000000015,
 5630.865,
 5639.846,
 5651.584,
 5703.853,
 5768.392,
 5772.8780000000015,
 5801.651,
 5803.271,
 5820.374,
 5826.733,
 583

Define a range for categories

In [88]:
lowest_category = (0, 3400)
low_category = (3400, 17000)
medium_category = (17000, 50000)
high_category = (50000, 100000)
highest_category = (100000, float('inf'))

Create a function to categorize total_income

In [89]:
def categorize_income(total_income):
    if total_income >= lowest_category[0] and total_income < lowest_category[1]:
        return 'Very Low'
    elif total_income >= low_category[0] and total_income < low_category[1]:
        return 'Low'
    elif total_income >= medium_category[0] and total_income < medium_category[1]:
        return 'Intermediate'
    elif total_income >= high_category[0] and total_income < high_category[1]:
        return 'High'
    elif total_income >= highest_category[0] and total_income < highest_category[1]:
        return 'Very high'
    else:
        return 'Uncategorized'

Apply the function with the data

In [90]:
data['income_category'] = data['total_income'].apply(categorize_income)

data[['total_income', 'income_category']].head()

,total_income,income_category
0,40620.102,Intermediate
1,17932.802,Intermediate
2,23341.752,Intermediate
3,42820.568,Intermediate
4,25378.572,Intermediate


Calculate the default percentage based on income level

In [92]:
income_debt_percentage = data.groupby('income_category')['debt'].mean().reset_index()

income_debt_percentage['debt'] = income_debt_percentage['debt'] * 100

income_debt_percentage

,income_category,debt
0,High,7.043407
1,Intermediate,8.312663
2,Low,7.823586
3,Very Low,50.000000
4,Very high,6.060606


Based on the observations above, the highest default rate is held by the category group with very low income with a figure of 50.00%. In other words, there is a correlation between income level and loan default. From the analysis above, people with the 'very high' income category have the smallest risk of defaulting on loan payments with a figure of 6.06%.

### Hypothesis 4: How do credit goals affect default percentage?

In [93]:
purpose_debt_percentage = data.groupby('purpose')['debt'].mean().reset_index()

purpose_debt_percentage['debt'] = purpose_debt_percentage['debt'] * 100

purpose_debt_percentage

,purpose,debt
0,building a property,8.723748
1,building a real estate,7.692308
2,buy commercial real estate,7.110439
3,buy real estate,6.924316
4,buy residential real estate,6.765677
5,buying a second-hand car,7.531381
6,buying my own car,9.108911
7,buying property for renting out,7.987711
8,car,8.502024
9,car purchase,9.110629


In [94]:
unique_purpose = data['purpose'].unique().tolist()

unique_purpose.sort()

unique_purpose

['building a property',
 'building a real estate',
 'buy commercial real estate',
 'buy real estate',
 'buy residential real estate',
 'buying a second-hand car',
 'buying my own car',
 'buying property for renting out',
 'car',
 'car purchase',
 'cars',
 'construction of own property',
 'education',
 'getting an education',
 'getting higher education',
 'going to university',
 'having a wedding',
 'housing',
 'housing renovation',
 'housing transactions',
 'profile education',
 'property',
 'purchase of a car',
 'purchase of my own house',
 'purchase of the house',
 'purchase of the house for my family',
 'real estate transactions',
 'second-hand car purchase',
 'supplementary education',
 'to become educated',
 'to buy a car',
 'to get a supplementary education',
 'to have a wedding',
 'to own a car',
 'transactions with commercial real estate',
 'transactions with my real estate',
 'university education',
 'wedding ceremony']

Create a dictionary to replace specific credit goals with broader categories

In [95]:
purpose_categories = {
    'building a property': 'property',
    'building a real estate': 'property',
    'buy commercial real estate': 'property',
    'buy real estate': 'property',
    'buy residential real estate': 'property',
    'buying a second-hand car': 'car',
    'buying my own car': 'car',
    'buying property for renting out': 'property',
    'car purchase': 'car',
    'cars': 'car',
    'construction of own property': 'property',
    'education': 'education',
    'getting an education': 'education',
    'getting higher education': 'education',
    'going to university': 'education',
    'having a wedding': 'wedding',
    'housing': 'property',
    'housing renovation': 'property',
    'housing transactions': 'property',
    'profile education': 'education',
    'property': 'property',
    'purchase of a car': 'car',
    'purchase of my own house': 'property',
    'purchase of the house': 'property',
    'purchase of the house for my family': 'property',
    'real estate transactions': 'property',
    'second-hand car purchase': 'car',
    'supplementary education': 'education',
    'to become educated': 'education',
    'to buy a car': 'car',
    'to get a supplementary education': 'education',
    'to have a wedding': 'wedding',
    'to own a car': 'car',
    'transactions with commercial real estate': 'property',
    'transactions with my real estate': 'property',
    'university education': 'education',
    'wedding ceremony': 'wedding'
}

Apply category replacement to the credit destination column

In [96]:
data['purpose_category'] = data['purpose'].map(purpose_categories)

In [97]:
data['purpose_category'].unique()

array(['property', 'car', 'education', 'wedding', nan], dtype=object)

Calculate the default percentage based on credit goals

In [98]:
purpose_debt_percentage = data.groupby('purpose_category')['debt'].mean().reset_index()

purpose_debt_percentage['debt'] = purpose_debt_percentage['debt'] * 100

purpose_debt_percentage

,purpose_category,debt
0,car,9.470094
1,education,9.220035
2,property,7.233373
3,wedding,8.003442


Based on the manipulation and observations above, credit objectives have an effect on the percentage of loan defaults. The goal categories 'car' and 'education' have a higher percentage than other categories. This indicates that people who take out loans for cars or education have a higher risk of default than other categories of purposes.

# General conclusion

To conclude the data analysis above, there are many processes that need to be carried out, such as dealing with missing data, transformation and manipulation, before analyzing the given hypothesis. 

For more details like this:
- Identification and handling of missing values: We found that the columns "days_employed" and "total_income" contained many missing values. We use the median to replace missing values ​​in these two columns.
- Identification and handling of suspicious data: We found some suspicious data, such as impossible age values, and we solved this problem by replacing them with reasonable values.
- Duplicate identification and handling: We found 71 data entries that were duplicates and removed them from the dataset.
- Data categorization: We categorize data based on several factors such as number of children, family status, income level, and credit goals to facilitate analysis.
- Correlation analysis: We carry out correlation analysis between factors such as number of children, family status, income level, and credit goals with the probability of loan default.

In the given hypothesis, all the questions above have a correlation with the probability of loan default. Even though the analysis above has validation and proof of how the correlation can be demonstrated, it must also be noted that this analysis can be used as an initial phase of analysis and the final results of this analysis can be influenced by other factors.